# Hello

![](./localhost.png)

In [1]:
# Import py2neo and connect to Neo4j
from py2neo import Graph

# just an example, replace with credentials for your own Neo4j instance
graph = Graph(bolt=True, host="localhost", http_port=7687, user='neo4j', password='kiss')

In [2]:
# Hello world, sanity check
graph.run("MATCH (a) RETURN COUNT(a) AS numberOfNodes").evaluate()

242654

In [ ]:
# # Create all the constraints

# graph.run("CREATE CONSTRAINT ON (p:Product) ASSERT p.id IS UNIQUE;")
# graph.run("CREATE CONSTRAINT ON (a:Aisle) ASSERT a.id IS UNIQUE;")
# graph.run("CREATE CONSTRAINT ON (d:Department) ASSERT d.id IS UNIQUE;")
# graph.run("CREATE CONSTRAINT ON (u:User) ASSERT u.id IS UNIQUE;")

In [ ]:
# # Load the CSV files
# # File are located at neo4j_home/import folder
# # products first

# graph.run("""
# USING PERIODIC COMMIT 
# LOAD CSV WITH HEADERS FROM "file:///products_clean.csv" AS line WITH line
# CREATE (product:Product {id: toInteger(line.product_id), name: line.product_name})
# MERGE (aisle:Aisle {id: toInteger(line.aisle_id), name: line.aisle})
# MERGE (department:Department {id: toInteger(line.department_id),name: line.department})
# CREATE (product)-[:FOUND_IN]->(aisle)
# CREATE (product)-[:TYPE_OF]->(department);
# """)

In [ ]:
# # users next
# # This will take awhile...

# graph.run("""
# USING PERIODIC COMMIT 
# LOAD CSV WITH HEADERS FROM "file:///users_orders.csv" AS line WITH line
# MATCH (product:Product {id: toInteger(line.product_id)})
# MERGE (user:User {id: toInteger(line.user_id)})
# CREATE (user)-[b:BOUGHT]->(product)
# SET b.order_total = toInteger(line.total_orders);
# """)

## Collaborative filtering

In [3]:
import pandas as pd

In [4]:
# Import number generator to generate a random user ID to query
from random import randint

In [ ]:
random_user = randint(0, 206210)
print("The selected user ID is: {}".format(random_user))

What if we go by aisle? What else do other users buy just as much or more in the same aisle?

In [ ]:
results = graph.run("""
MATCH (user:User {id: {user_id}})-[b1:BOUGHT]->()-[:FOUND_IN]->()<-[:FOUND_IN]-(otherproduct:Product)<-[b2:BOUGHT]-(otheruser:User)
WHERE NOT (user)-[:BOUGHT]->(otherproduct) AND b2.order_total >= b1.order_total
RETURN otherproduct.name AS recommendation,
       COUNT(*) AS usersInCommon
ORDER BY usersInCommon DESC
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

## Simple Version (the KNN of Recommendations)

What else do other users buy just as much as?

In [ ]:
results = graph.run("""
MATCH (user:User {id: {user_id}})-[b1:BOUGHT]->(:Product)<-[b2:BOUGHT]-(otheruser:User)
MATCH (otheruser)-[:BOUGHT]->(rec:Product)
WHERE NOT EXISTS( (user)-[:BOUGHT]->(rec)) AND b2.order_total >= b1.order_total
RETURN rec.name, COUNT(*) AS usersWhoAlsoBought
ORDER BY usersWhoAlsoBought DESC 
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

Consider the type of food.

In [12]:
random_user = randint(0, 206210)
print("Next user is: {}".format(random_user))

Next user is: 159738


In [ ]:
results = graph.run("""
MATCH (user:User {id: {user_id}})-[b:BOUGHT]->(p:Product)
WITH user, avg(b.order_total) AS mean

MATCH (user)-[b:BOUGHT]->(p:Product)-[:FOUND_IN]->(a:Aisle)
WHERE b.order_total >= mean

WITH user, a, COUNT(*) AS score

MATCH (a)<-[:FOUND_IN]-(rec:Product)
WHERE NOT EXISTS((user)-[:BOUGHT]->(rec))

RETURN rec.name AS recommendation, COLLECT(DISTINCT a.name) AS productType, SUM(score) AS sscore
ORDER BY sscore DESC LIMIT 10
""", user_id=random_user)

for row in results:
    print(row)

Not helpful with the same scores...

Let's see if similarity metrics will help instead.

### 1) Cosine Similarity

The cosine similarty of two users will tell us how similar two users' preferences for products are. Users with a high cosine similarity will have similar preferences.

In [ ]:
results = graph.run("""
MATCH (p1:User {id: {user_id}})-[x:BOUGHT]->(p:Product)<-[y:BOUGHT]-(p2:User)
WITH COUNT(p) AS numberproducts, SUM(x.order_total * y.order_total) AS xyDotProduct,
SQRT(REDUCE(xDot = 0.0, a IN COLLECT(x.order_total) | xDot + a^2)) AS xLength,
SQRT(REDUCE(yDot = 0.0, b IN COLLECT(y.order_total) | yDot + b^2)) AS yLength,
p1, p2 WHERE numberproducts > 10
RETURN p2.id, xyDotProduct / (xLength * yLength) AS cosim
ORDER BY cosim DESC 
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

### 2) Pearson Similarity

This is particularly well-suited for product recommendations because it takes into account the fact that different users will have different mean total orders: on average some people do buy only from Instacart, while some prefer to go out of their house. Since Pearson similarity considers differences about the mean, this metric will account for these discrepancies.

In [ ]:
results = graph.run("""
MATCH (u1:User {id: {user_id}})-[r:BOUGHT]->(m:Product)
WITH u1, avg(r.order_total) AS u1_mean

MATCH (u1)-[r1:BOUGHT]->(m:Product)<-[r2:BOUGHT]-(u2)
WITH u1, u1_mean, u2, COLLECT({r1: r1, r2: r2}) AS totalorders WHERE size(totalorders) > 10

MATCH (u2)-[r:BOUGHT]->(m:Product)
WITH u1, u1_mean, u2, avg(r.order_total) AS u2_mean, totalorders

UNWIND totalorders AS r

WITH sum( (r.r1.order_total - u1_mean) * (r.r2.order_total - u2_mean) ) AS nom,
     sqrt( sum( (r.r1.order_total - u1_mean)^2) * sum( (r.r2.order_total - u2_mean) ^2)) AS denom,
     u1, u2 WHERE denom <> 0

RETURN u2.id, nom/denom AS pearson
ORDER BY pearson DESC 
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

I have a problem with so many 0.90+ scores for the cosine similarity metric. The Pearson scores seems to be a better spread...

## FINAL: Collaborative Filtering = KNN + Pearson 

Combine the simple implementation of KNN with the Pearson correlation scoring system:

In [ ]:
results = graph.run("""
MATCH (u1:User {id: {user_id}})-[r:BOUGHT]->(m:Product)
WITH u1, avg(r.order_total) AS u1_mean

MATCH (u1)-[r1:BOUGHT]->(m:Product)<-[r2:BOUGHT]-(u2)
WITH u1, u1_mean, u2, COLLECT({r1: r1, r2: r2}) AS totalorders WHERE size(totalorders) > 10

MATCH (u2)-[r:BOUGHT]->(m:Product)
WITH u1, u1_mean, u2, avg(r.order_total) AS u2_mean, totalorders

UNWIND totalorders AS r

WITH sum( (r.r1.order_total - u1_mean) * (r.r2.order_total - u2_mean) ) AS nom,
     sqrt( sum( (r.r1.order_total - u1_mean)^2) * sum( (r.r2.order_total - u2_mean) ^2)) AS denom,
     u1, u2 WHERE denom <> 0

WITH u1, u2, nom/denom AS pearson
ORDER BY pearson DESC LIMIT 10

MATCH (u2)-[r:BOUGHT]->(m:Product) WHERE NOT EXISTS( (u1)-[:BOUGHT]->(m) )

RETURN m.name AS recommendation, SUM( pearson * r.order_total) AS score
ORDER BY score DESC 
LIMIT 10""", user_id=random_user)

for row in results:
    print(row)

#### **Conclusion**

I am quite happy using the Pearson similarity to find other products from similar users based on the total number of orders for that product in lieu of a ratings system.

## Content-based filtering

What are other products similar to what you are looking at? Let's bring in the categorical nature of some of the products. 

If we know what products a user has bought, we can use this information to recommend similar products: Recommend products similar to those the user has already bought.

In [ ]:
# Case sensitive... 'Bacon' worked but 'bacon' didn't work
# Case very sensitive... 'Product' works but not 'PRODUCT'

result = graph.run('''
MATCH (m:Product)-[:FOUND_IN]->(g:Aisle)<-[:FOUND_IN]-(rec:Product)
WHERE m.name CONTAINS 'Bacon'
WITH rec.name AS recommendation, g.name AS aisle, COUNT(*) AS commonAisles
RETURN recommendation, aisle, commonAisles
ORDER BY commonAisles DESC 
LIMIT 10''')

for row in result:
    print(row)

99 commonAisles is a problem. lol. What is we take a user and then recommend based on the types of products. Either by aisle or by department.

In [ ]:
random_user = randint(0, 206210)
print("Next user is: {}".format(random_user))

#### By Aisle

In [ ]:
result = graph.run('''
MATCH (u:User {id: {user_id}})-[r:BOUGHT]->(m:Product), (m)-[:FOUND_IN]->(g:Aisle)<-[:FOUND_IN]-(rec:Product)
WHERE NOT EXISTS( (u)-[:BOUGHT]->(rec) )
WITH rec.name AS recommendation, [g.name, COUNT(*)] AS scores
RETURN recommendation, COLLECT(scores) AS scoreComponents, REDUCE (s=0,x in COLLECT(scores) | s+x[1]) AS score
ORDER BY score DESC 
LIMIT 10''', user_id=random_user)

for row in result:
    print(row)

#### By Department

In [ ]:
result = graph.run('''
MATCH (u:User {id: {user_id}})-[r:BOUGHT]->(m:Product), (m)-[:TYPE_OF]->(g:Department)<-[:TYPE_OF]-(rec:Product)
WHERE NOT EXISTS( (u)-[:BOUGHT]->(rec) )
WITH rec.name AS recommendation, [g.name, COUNT(*)] AS scores
RETURN recommendation, COLLECT(scores) AS scoreComponents, REDUCE (s=0,x in COLLECT(scores) | s+x[1]) AS score
ORDER BY score DESC 
LIMIT 10''', user_id=random_user)

for row in result:
    print(row)

Separately (Aisle vs Department), I think they are pretty rubbish recommendations. Let's try a weighted sum of these two components in the 'score'.

In [ ]:
result = graph.run('''
MATCH (m:Product) WHERE m.name CONTAINS 'Bacon'
MATCH (m)-[:FOUND_IN]->(g:Aisle)<-[:FOUND_IN]-(rec:Product)

WITH m, rec, COUNT(*) AS gs

OPTIONAL MATCH (m)-[:TYPE_OF]->(a:Department)<-[:TYPE_OF]-(rec)
WITH m, rec, gs, COUNT(*) AS as

RETURN rec.name AS recommendation, (5*gs)+(3*as) AS score ORDER BY score DESC 
LIMIT 10
''')

for row in result:
    print(row)

If we add a minimum order total of at least 5, the query takes a long time to run...

In [ ]:
# Will take awhile

result = graph.run('''
MATCH (m:Product) WHERE m.name CONTAINS 'Bacon'
MATCH (m)-[:FOUND_IN]->(g:Aisle)<-[:FOUND_IN]-(rec:Product)

WITH m, rec, COUNT(*) AS gs

OPTIONAL MATCH (m)-[:TYPE_OF]->(a:Department)<-[:TYPE_OF]-(rec)
WITH m, rec, gs, COUNT(*) AS as

OPTIONAL MATCH (m)<-[b:BOUGHT]-(d:User) WHERE b.order_total > 5
WITH m, rec, gs, as, COUNT(d) AS ds

RETURN rec.name AS recommendation, (5*gs)+(3*as)+(1*ds) AS score 
ORDER BY score DESC 
LIMIT 10
''', user_id=random_user)

for row in result:
    print(row)

But everything is "stuck" in the same category/type versus the previous one.

In [ ]:
result = graph.run('''
MATCH (m:Product) WHERE m.name CONTAINS 'Bacon'
MATCH (m)-[:FOUND_IN]->(g:Aisle)<-[:FOUND_IN]-(rec:Product)

WITH m, rec, COUNT(*) AS gs

OPTIONAL MATCH (m)-[:TYPE_OF]->(a:Department)<-[:TYPE_OF]-(rec)
WITH m, rec, gs, COUNT(a) AS as

RETURN rec.name AS recommendation, (5*gs)+(3*as) AS score ORDER BY score DESC 
LIMIT 10
''')

for row in result:
    print(row)

So far, we used 'common traits' to traverse the network and return results. As we can see the 'scores' for most of these are the same. No doubt they seem to produce the same order but for reproducibility and for robustness, we need a form of similarity scoring.

### 1) Jaccard Index

From [Wikipedia](https://en.wikipedia.org/wiki/Jaccard_index): The Jaccard index, also known as Intersection over Union and the Jaccard similarity coefficient (originally coined coefficient de communauté by Paul Jaccard), is a statistic used for comparing the similarity and diversity of sample sets. The Jaccard coefficient measures similarity between finite sample sets, and is defined as the size of the intersection divided by the size of the union of the sample sets.

**TL;DR** The Jaccard index is a number between 0 and 1 that indicates how similar two sets are. The Jaccard index of two identical sets is 1. If two sets do not have a common element, then the Jaccard index is 0. 

We can calculate the Jaccard index for sets of product via their aisles to determine how similar two products are.

In [ ]:
random_product = randint(0, 49687)
print("Random product is: {}".format(random_product))

In [ ]:
product = graph.run('''
MATCH (m:Product {id: {product_id}})
RETURN m.name
''', product_id=random_product)

for row in product:
    print(row)

In [ ]:
result = graph.run('''
MATCH (m:Product {id: {product_id}})-[:FOUND_IN]->(a:Aisle)<-[:FOUND_IN]-(other:Product)
WITH m, other, COUNT(a) AS intersection, COLLECT(a.name) AS i

MATCH (m)-[:FOUND_IN]->(ma:Aisle)
WITH m, other, intersection,i, COLLECT(ma.name) AS s1

MATCH (other)-[:FOUND_IN]->(oa:Aisle)
WITH m, other,intersection,i, s1, COLLECT(oa.name) AS s2

WITH m, other,intersection, s1,s2

WITH m, other,intersection, s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

RETURN other.name AS recommendation, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC 
LIMIT 10
''', product_id=random_product)

for row in result:
    print(row)

In [ ]:
result = graph.run('''
MATCH (m:Product {id: {product_id}})-[:TYPE_OF]->(d:Department)<-[:TYPE_OF]-(other:Product)
WITH m, other, COUNT(d) AS intersection, COLLECT(d.name) AS i

MATCH (m)-[:TYPE_OF]->(md:Department)
WITH m, other, intersection,i, COLLECT(md.name) AS s1

MATCH (other)-[:TYPE_OF]->(od:Department)
WITH m, other,intersection,i, s1, COLLECT(od.name) AS s2

WITH m, other,intersection, s1,s2

WITH m, other,intersection, s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

RETURN other.name AS recommendation, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC 
LIMIT 10
''', product_id=random_product)

for row in result:
    print(row)

Once again, separating the aisle and department, will just yield a 1.0 Jaccard score, because they _are_ the same. So let's mix the two up.

In [ ]:
result = graph.run('''
MATCH (m:Product {id: {product_id}})-[:TYPE_OF|:FOUND_IN]-(t)<-[:TYPE_OF|:FOUND_IN]-(other:Product)
WITH m, other, COUNT(t) AS intersection, COLLECT(t.name) AS i

MATCH (m)-[:TYPE_OF|:FOUND_IN]-(mt)
WITH m, other, intersection,i, COLLECT(mt.name) AS s1

MATCH (other)-[:TYPE_OF|:FOUND_IN]-(ot)
WITH m, other,intersection,i, s1, COLLECT(ot.name) AS s2

WITH m, other,intersection,s1,s2

WITH m, other,intersection,s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

RETURN other.name AS recommendation, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC
LIMIT 10''', product_id=random_product)

for row in result:
    print(row)

In [ ]:
result = graph.run('''
MATCH (m:Product {id: {product_id}})-[:TYPE_OF|:FOUND_IN]-(t)<-[:TYPE_OF|:FOUND_IN]-(other:Product)
WITH m, other, COUNT(t) AS intersection, COLLECT(t.name) AS i

MATCH (m)-[:TYPE_OF|:FOUND_IN]-(mt)
WITH m, other, intersection,i, COLLECT(mt.name) AS s1

MATCH (other)-[:TYPE_OF|:FOUND_IN]-(ot)
WITH m, other,intersection,i, s1, COLLECT(ot.name) AS s2

WITH m, other,intersection,s1,s2

WITH m, other,intersection,s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2

RETURN other.name AS recommendation, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard ASC
LIMIT 10''', product_id=random_product)

for row in result:
    print(row)

I realised that Jaccard scores will be whole numbers, because for each product it can be found in one aisle and one department respectively. It doesn't have multiple aisles or multiple departments. So, I now turn to clustering to see if I can use it for something.

### 2) Clustering

In [7]:
from igraph import Graph as IGraph

In [13]:
result = graph.run('''
MATCH (a:Aisle)<-[:FOUND_IN]-()-[:TYPE_OF]->(d:Department)
WHERE ID(a) < ID(d)
RETURN a.name AS aisleName, d.name AS departmentName, COUNT(*) AS weight
ORDER BY weight DESC
LIMIT 10''')

for row in result:
    print(row)

('aisleName': 'yogurt', 'departmentName': 'dairy eggs', 'weight': 1026)
('aisleName': 'tea', 'departmentName': 'beverages', 'weight': 891)
('aisleName': 'frozen meals', 'departmentName': 'frozen', 'weight': 880)
('aisleName': 'cookies cakes', 'departmentName': 'snacks', 'weight': 874)
('aisleName': 'spices seasonings', 'departmentName': 'pantry', 'weight': 797)
('aisleName': 'packaged vegetables fruits', 'departmentName': 'produce', 'weight': 615)
('aisleName': 'asian foods', 'departmentName': 'international', 'weight': 604)
('aisleName': 'other', 'departmentName': 'other', 'weight': 546)
('aisleName': 'canned jarred vegetables', 'departmentName': 'canned goods', 'weight': 487)
('aisleName': 'cereal', 'departmentName': 'breakfast', 'weight': 454)


In [14]:
cluster = graph.run('''
MATCH (a:Aisle)<-[:FOUND_IN]-()-[:TYPE_OF]->(d:Department)
WHERE ID(a) < ID(d)
RETURN a.name AS aisleName, d.name AS departmentName, COUNT(*) AS weight''')

In [15]:
ig = IGraph.TupleList(cluster, weights=True)
ig

In [16]:
clusters = IGraph.community_walktrap(ig, weights='weight')
clusters = clusters.as_clustering()
len(clusters)

16

In [18]:
# Let's take a look at the 'clusters'
nodes = [node['name'] for node in ig.vs]
nodes = [{'id': x, 'label': x} for x in nodes]
nodes[:5]

for node in nodes:
    idx = ig.vs.find(name=node['id']).index
    node['group'] = clusters.membership[idx]
    
nodes[:20]

[{'group': 0,
  'id': 'specialty wines champagnes',
  'label': 'specialty wines champagnes'},
 {'group': 0, 'id': 'alcohol', 'label': 'alcohol'},
 {'group': 1, 'id': 'spices seasonings', 'label': 'spices seasonings'},
 {'group': 1, 'id': 'pantry', 'label': 'pantry'},
 {'group': 2, 'id': 'cereal', 'label': 'cereal'},
 {'group': 2, 'id': 'breakfast', 'label': 'breakfast'},
 {'group': 3, 'id': 'poultry counter', 'label': 'poultry counter'},
 {'group': 3, 'id': 'meat seafood', 'label': 'meat seafood'},
 {'group': 4, 'id': 'frozen meals', 'label': 'frozen meals'},
 {'group': 4, 'id': 'frozen', 'label': 'frozen'},
 {'group': 5,
  'id': 'packaged vegetables fruits',
  'label': 'packaged vegetables fruits'},
 {'group': 5, 'id': 'produce', 'label': 'produce'},
 {'group': 6, 'id': 'tea', 'label': 'tea'},
 {'group': 6, 'id': 'beverages', 'label': 'beverages'},
 {'group': 7, 'id': 'cookies cakes', 'label': 'cookies cakes'},
 {'group': 7, 'id': 'snacks', 'label': 'snacks'},
 {'group': 8, 'id': 'yog

In [ ]:
# # Write it back into the database
# #
# # Create constraint on cluster name

# graph.run("CREATE CONSTRAINT ON (cluster:Cluster) ASSERT cluster.name IS UNIQUE;")

In [ ]:
# # Write it back into the database
# #
# # Writing Aisle first

# graph.run('''
# UNWIND {params} AS p 
# MATCH (a:Aisle {name: p.id})
# MERGE (cluster:Cluster {name: p.group})
# MERGE (a)-[:IN_CLUSTER]->(cluster)
# ''', params = nodes)

In [ ]:
# # Write it back into the database
# #
# # Writing Department next

# graph.run('''
# UNWIND {params} AS p 
# MATCH (d:Department {name: p.id})
# MERGE (cluster:Cluster {name: p.group})
# MERGE (d)-[:IN_CLUSTER]->(cluster)
# ''', params = nodes)

In [ ]:
random_product = randint(0, 49687)
print("Random product is: {}".format(random_product))

### Novelty Recommendation

I found that having clusters would allow me to group aisles together. A good use for this is to find something for "Try Something New" but not too foreign.

In [ ]:
# This is giving me stuff on the opposite end of what a user orders most often
# Is this my novelty recommendation?

MATCH (user:User {id:159738})-[:BOUGHT]->(product)-[:FOUND_IN]->(a:Aisle)-[:IN_CLUSTER]->(cluster)
WITH user, cluster, COUNT(*) AS times
ORDER BY times DESC
LIMIT 1
WITH user, cluster
MATCH (cluster)<-[:IN_CLUSTER]-(a)<-[:FOUND_IN]-()
WITH cluster, a.name AS aisles, COUNT(a) as commonAisles
ORDER BY commonAisles ASC
LIMIT 1
WITH aisles AS x
MATCH (Aisle {name: x })<-[:FOUND_IN]-(otherProducts)<-[b:BOUGHT]-()
WHERE b.order_total > 10
RETURN DISTINCT otherProducts.name, b.order_total
ORDER BY b.order_total DESC
LIMIT 10

## Making recommendations

What to buy next? Meat -> Vegetable -> Wine or other way round